# Загрузка данных и EDA

In [6]:
import pandas as pd

# Загружаем датасет
df = pd.read_csv("data/lighting_dataset.csv")

print("Размер датасета:", df.shape)

# 1. Проверка на полные дубликаты строк
duplicates = df.duplicated().sum()
print(f"Полных дубликатов строк: {duplicates}")

# 2. Проверка дубликатов по названию модели
model_dupes = df["model_name"].duplicated().sum()
print(f"Повторяющихся названий моделей: {model_dupes}")

# 3. Кол-во уникальных моделей
unique_models = df["model_name"].nunique()
print(f"Уникальных моделей: {unique_models}")

# 4. Топ-10 самых повторяющихся моделей
print("\nТоп-10 дубликатов model_name:")
print(df["model_name"].value_counts().head(10))

# 5. Проверка на NaN
print("\nПропуски по колонкам:")
print(df.isna().sum())


Размер датасета: (20000, 15)
Полных дубликатов строк: 0
Повторяющихся названий моделей: 0
Уникальных моделей: 20000

Топ-10 дубликатов model_name:
model_name
OptiLight BE18101    1
EcoLight MU12690     1
ZetaLED AT12042      1
Osram BO14006        1
MarketLED QO13325    1
Oculus KV5381        1
MarketLED NR6981     1
Зенит KS7718         1
OptiLight BF292      1
ПроСвет VN7144       1
Name: count, dtype: int64

Пропуски по колонкам:
room_type         0
segment           0
area_m2           0
ceiling_h         0
budget            0
required_lux      0
total_lumens      0
fixtures_count    0
fixture_type      0
fixture_lm        0
purpose           0
country           0
model_name        0
price_rub         0
norm_ref          0
dtype: int64


In [7]:
df.head(10)

,room_type,segment,area_m2,ceiling_h,budget,required_lux,total_lumens,fixtures_count,fixture_type,fixture_lm,purpose,country,model_name,price_rub,norm_ref
0,Выставочный зал,administrative,673,11.8,348390,502,486498,145,Подвесной,3339,Выставочный зал,США,OptiLight BE18101,2387,СП 52.13330.2016
1,Парковка,outdoor,1977,0.0,146010,51,100827,8,Прожектор,11435,Парковка,Франция,StreetLine AB17479,12578,СП 52.13330.2016
2,Больница (палата),administrative,15,5.9,1537,186,3194,2,Настенный линейный,1169,Больница (палата),Тайвань,ProLED HV8360,578,СП 52.13330.2016
3,Гостиная,domestic,31,2.9,1596,192,5952,1,Подвесной,3029,Гостиная,Тайвань,ЭРА BF5296,1575,СП 52.13330.2016
4,Жилое помещение,domestic,13,2.9,656,198,2574,2,Настенный (бра),895,Жилое помещение,Корея,HomeLight HA3443,268,СанПиН 2.2.1
5,Цех (ресторан/столовая),administrative,134,9.8,84762,271,48660,3,Промышленный (High-bay),15894,Цех (ресторан/столовая),Германия,ПроСвет JN9174,19867,СП 52.13330.2016
6,Сцена театра,administrative,238,9.6,303636,1061,335848,93,Подвесной,3578,Сцена театра,Корея,OptiLight IS7534,2558,СНиП 23-05-95
7,Парковка,outdoor,466,0.0,31333,48,22368,2,Консольный,10096,Парковка,Россия,Андромеда ZY12451,14437,СП 52.13330.2016
8,Производственные (точные работы),industrial,270,8.9,121349,455,159090,50,Встраиваемый,3125,Производственные (точные работы),Россия,Феникс XP17909,2250,СП 52.13330.2016
9,Санузел,domestic,8,2.7,1922,49,392,1,Потолочный,3199,Санузел,Турция,LumenPro FV9418,1759,СП 52.13330.2016


# Модель LightGBM + Optuna


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
from lightgbm import LGBMRegressor
import optuna
import joblib

# Загружаем датасет
df = pd.read_csv("data/lighting_dataset.csv")

# --- Целевая переменная для ML ---
X = df[["area_m2", "ceiling_h", "required_lux", "fixture_lm"]].astype("float64")
y = df["fixtures_count"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- Оптимизация гиперпараметров Optuna ---
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 31, 255),
        "max_depth": trial.suggest_int("max_depth", -1, 20),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "random_state": 42,
        "n_jobs": -1,
        "verbose": -1
    }

    model = LGBMRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    # применяем нижнюю границу ≥ 1
    preds = [max(1, round(p)) for p in preds]
    rmse = root_mean_squared_error(y_test, preds)
    return rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Лучшие параметры:", study.best_params)

# --- Финальная модель ---
best_params = study.best_params
final_model = LGBMRegressor(**best_params)
final_model.fit(X_train, y_train)

# --- Проверка ---
preds = final_model.predict(X_test)
preds = [max(1, round(p)) for p in preds]

rmse = root_mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)

print(f"Fixtures → RMSE={rmse:.3f}, R²={r2:.3f}")

[I 2025-09-21 19:20:09,519] A new study created in memory with name: no-name-fe1e8363-beef-479e-ab4c-79d4266a3835
[I 2025-09-21 19:20:11,058] Trial 0 finished with value: 2.944189192290468 and parameters: {'n_estimators': 668, 'learning_rate': 0.02871680625321858, 'num_leaves': 131, 'max_depth': 19, 'subsample': 0.9864778345871564, 'colsample_bytree': 0.6726453420461315}. Best is trial 0 with value: 2.944189192290468.
[I 2025-09-21 19:20:11,878] Trial 1 finished with value: 3.002290792045301 and parameters: {'n_estimators': 917, 'learning_rate': 0.011447279904020204, 'num_leaves': 40, 'max_depth': 11, 'subsample': 0.9455706982085778, 'colsample_bytree': 0.7235938538545421}. Best is trial 0 with value: 2.944189192290468.
[I 2025-09-21 19:20:11,956] Trial 2 finished with value: 12.743763180473811 and parameters: {'n_estimators': 292, 'learning_rate': 0.02135099308128157, 'num_leaves': 56, 'max_depth': 2, 'subsample': 0.708170476862441, 'colsample_bytree': 0.961397613522316}. Best is tria

Лучшие параметры: {'n_estimators': 951, 'learning_rate': 0.1241869240467792, 'num_leaves': 249, 'max_depth': 5, 'subsample': 0.755525127621188, 'colsample_bytree': 0.966787051046508}
Fixtures → RMSE=2.020, R²=0.998


In [2]:
# --- Сохранение модели ---
joblib.dump(final_model, "models/fixtures_model.pkl")
print("Модель сохранена: models/fixtures_model.pkl")

Модель сохранена: models/fixtures_model.pkl
